In [1]:
import pandas as pd
import numpy as np
import nltk
import json
from simpletransformers.ner import NERModel
from gensim.models import KeyedVectors
from strsimpy.jaro_winkler import JaroWinkler
from spellchecker import SpellChecker
from nltk.corpus import stopwords

In [2]:
%run constants.ipynb
%run functions.ipynb
%run dictionnaries.ipynb

In [3]:
%run tag_to_filter.v1.ipynb

In [4]:
df_transitions = pd.read_csv(bdd_directory+'df_transitions.csv', 
                             sep='§', 
                             engine='python',
                             index_col=0, 
                             encoding='utf-8')
df_transitions.shape

(10, 10)

In [5]:
class Markov:
    def __init__(self, name, transitions):
        self.name = name
        self.transitions = transitions
        
    def predict(self, event):
        predict = predict_next_state(event, self.transitions)
        return predict
    
    def random_state(self):
        return np.random.choice(self.transitions.index)
    
markov = Markov('Markov', df_transitions)
markov.predict(markov.random_state())

'{"filters": {"manufacturer": ["airbus"], "aircraft": ["a320"], "company": [], "category": [], "date": ["01092017", "30092017"]}, "tab": "seat"}'

### Loads NER and W2V models 
source for the NER model : https://ufile.io/ichyycfe  
source for the W2V model : https://github.com/eyaler/word2vec-slim/blob/master/GoogleNews-vectors-negative300-SLIM.bin.gz  

In [6]:
all_tags = get_all_tags()
tagger = NERModel(model_type = 'bert',
                  model_name = data_directory+'bert/current_model',
                  labels = all_tags,
                  use_cuda = False)

model_w2v = KeyedVectors.load_word2vec_format(pathword2vec, binary=True)
voc_stopwords = set(stopwords.words('english'))
db = get_BDD()

In [7]:
class Intent:
    def __init__(self, name):
        self.name = name

class Afficheur(Intent):
    def __init__(self, name, model_tagger, model_predicteur, model_w2v, db={}, voc_stopwords=set()):
        self.name = name
        self.tagger = model_tagger
        self.predicteur = model_predicteur
        self.w2v = model_w2v
        self.voc_stopwords = voc_stopwords
        self.db = db
    
    def get_tags(self, sentence):
        sentence_corrected_list = self.auto_correction(sentence)
        sentence_corrected = " ".join(sentence_corrected_list)
        request = self.tagger.predict([sentence_corrected])[0][0]
        print(request,'\n')
        return request
    
    def get_filters(self, sentence):
        request = self.get_tags(sentence)
        tags_values = extract_tags(request)
        filters = tag_to_filters(tags_values)
        filters = apply_date(filters)
        event = {
            CT_tabs : CT_tabs_default,
            CT_filt : filters,
        }
        return event
    
    def interact(self, sentence):
        event = self.get_filters(sentence)
        event_state = json.dumps(event)
        event_state_hash = json_string_to_hash(event_state)
        pred_state_hash = self.predicteur.predict(event_state_hash)
        pred_sentence = make_sentence_fom_json(json.loads(pred_state_hash))
        
        res = {"in_sate" : event_state_hash,
               "out_state" : pred_state_hash,
               "out_sentence" : pred_sentence,}
        return res
    
    def auto_correction(self, sentence):
        return auto_correction(self.w2v.vocab, db, voc_stopwords, sentence)
    
Afficheur_Intent = Afficheur(name='Afficheur', 
                             model_tagger=tagger, 
                             model_predicteur=markov,
                             model_w2v=model_w2v,
                             voc_stopwords=voc_stopwords,
                             db=db)
Afficheur_Intent.name

'Afficheur'

In [8]:
class Classifieur:
    def __init__(self, name, model_classifieur):
        self.name = name
        self.classifieur = model_classifieur
        
    def predict(self, sentence):
        return 0

Classifieur = Classifieur(name='Intent classifieur', model_classifieur="")
Classifieur.predict(sentence="")

0

In [9]:
intents  = {"classifieur" : Classifieur,
            "intents" : {0 : Afficheur_Intent}
           }

intents

{'classifieur': <__main__.Classifieur at 0x7f1c7a3f58d0>,
 'intents': {0: <__main__.Afficheur at 0x7f1c7a3f56d8>}}

In [10]:
class ChatBot:
    def __init__(self, name, intents={}, classifieur=""):
        self.name = name
        self.classifieur = intents['classifieur']
        self.intents = intents['intents']
        
    def get_intent_classification(self, sentence):
        return self.classifieur.predict(sentence)
    
    def interact(self, sentence):
        intent_id = self.get_intent_classification(sentence)
        intent = self.intents[intent_id]
        interaction = intent.interact(sentence)
        return interaction

    
    
Hubert = ChatBot(name='Chatbot Hubert', intents=intents)
Hubert.get_intent_classification('aertetreb')

0

In [11]:
res = Hubert.interact('show me numberd of flight in FRAnCE for Airbus')
for k,v in res.items():
    print(k,':\t',v,'\n')

Converting to features started.



[{'show': 'O'}, {'me': 'O'}, {'number': 'STUDIED_B'}, {'of': 'STUDIED_I'}, {'flight': 'STUDIED_E'}, {'in': 'O'}, {'France': 'COUN_B'}, {'for': 'O'}, {'Airbus': 'MANU_B'}] 

in_sate :	 {"filters": {"manufacturer": ["airbus"], "aircraft": [], "company": [], "country": ["france"], "category": [], "date": []}, "tab": "default"} 

out_state :	 {"filters": {"manufacturer": ["airbus"], "aircraft": ["a350-900"], "company": ["air france"], "category": [], "date": ["01092017", "30092017"]}, "tab": "general"} 

out_sentence :	 We suggest you thegenerals' study from 01-09-2017 to 30-09-2017 for the manufacturerairbus and for the aircrafta350-900 and for the companyair france and. If you agree, click onto the following link ;) 

